<!-- <h2>Title</h2> -->

---
<h1> 
    National Income Dynamics Study-Coronavirus Rapid Mobile Survey (NIDS-CRAM)<br>
    Wave 1: February - April 2020<br><br>
    The Effect of COVID-19 on the Labour and Welfare of South Africans
</h1>

--- 


<h1> 
Part 2 - Rough Draft
</h1>

--- 


<h2>Group Members</h2>
Dino Anastasopoulos: 1900661 - Contributed towards the Demographics section. <br>
Brenton Budler: 1827655 - Contributed towards the Labour section. <br>
Philani Mpofu: 1848751 - Contributed towards the Welfare section. <br>

--- 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Group-Members" data-toc-modified-id="Group-Members-1">Group Members</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-2">Imports</a></span></li><li><span><a href="#Read-data" data-toc-modified-id="Read-data-3">Read data</a></span></li><li><span><a href="#Model" data-toc-modified-id="Model-4">Model</a></span><ul class="toc-item"><li><span><a href="#Drop-all-nans" data-toc-modified-id="Drop-all-nans-4.1">Drop all nans</a></span></li><li><span><a href="#Now-this-is-the-dataframe-of-all-people-who-kept-their-jobs,-and-have-usable-data" data-toc-modified-id="Now-this-is-the-dataframe-of-all-people-who-kept-their-jobs,-and-have-usable-data-4.2">Now this is the dataframe of all people who kept their jobs, and have usable data</a></span></li><li><span><a href="#Calculate-and-store-salary-difference" data-toc-modified-id="Calculate-and-store-salary-difference-4.3">Calculate and store salary difference</a></span></li><li><span><a href="#Removing-outliers" data-toc-modified-id="Removing-outliers-4.4">Removing outliers</a></span></li><li><span><a href="#Determine-if-salary-changed-and-create-new-variable-for-this" data-toc-modified-id="Determine-if-salary-changed-and-create-new-variable-for-this-4.5">Determine if salary changed and create new variable for this</a></span></li><li><span><a href="#Encode-some-variables-and-get-dummies" data-toc-modified-id="Encode-some-variables-and-get-dummies-4.6">Encode some variables and get dummies</a></span></li><li><span><a href="#Store-everything-done-in-a-new-dataframe-and-drop-columns-that-we-don't-want-in-model" data-toc-modified-id="Store-everything-done-in-a-new-dataframe-and-drop-columns-that-we-don't-want-in-model-4.7">Store everything done in a new dataframe and drop columns that we don't want in model</a></span></li><li><span><a href="#Check-balance-of-classes-that-are-being-predicted" data-toc-modified-id="Check-balance-of-classes-that-are-being-predicted-4.8">Check balance of classes that are being predicted</a></span></li><li><span><a href="#Logistic-regression" data-toc-modified-id="Logistic-regression-4.9">Logistic regression</a></span></li></ul></li></ul></div>

## Imports

In [98]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

import matplotlib.cm as cm

from matplotlib.colors import Normalize

import statsmodels.formula.api as smf
import statsmodels.api as sm


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.preprocessing import StandardScaler, LabelEncoder

print("Successfully imported all")


Successfully imported all


## Read data

In [87]:
dem = pd.read_csv("Demographics.csv",)
lab = pd.read_csv("Labour.csv")

df = pd.concat([dem,lab],axis=1)

## Model

**Looking at what factors determined a change in a persons salary**

In [88]:
dftemp = pd.concat([dem,lab],axis=1)

In [89]:
display(dftemp)
dftemp.shape

,age,gender,race,highest_grade,tertiary_edu,province_current,province_moved,province_before,province_during,labour_in_feb,...,work_hours_feb,take_home_pay_feb,labour_in_apr,work_days_apr,work_hours_apr,take_home_pay_apr,lost_labour,kept_labour,return_to_work,usual_work
0,39.0,Man,White,Grade 9,No,Gauteng,No,NaN,NaN,1,...,NaN,180.0,0,0.0,0.0,0.0,1,0,0,Unknown
1,72.0,Woman,Asian/Indian,Grade 0,NaN,KwaZulu-Natal,No,NaN,NaN,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
2,30.0,Woman,African/Black,Grade 10,Yes,Gauteng,No,NaN,NaN,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
3,48.0,Woman,African/Black,Grade 10,No,Gauteng,No,NaN,NaN,1,...,5.0,900.0,0,0.0,0.0,0.0,1,0,1,Elementary occupations
4,49.0,Woman,African/Black,Grade 9,No,Gauteng,No,NaN,NaN,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7068,22.0,Man,African/Black,Grade 11,No,Northern Cape,No,NaN,NaN,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
7069,24.0,Woman,Coloured,Grade 10,No,Western Cape,No,NaN,NaN,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
7070,36.0,Woman,African/Black,NTC 3,No,North West,No,NaN,NaN,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed
7071,20.0,Man,African/Black,Grade 12,No,North West,Yes,Gauteng,Gauteng,0,...,0.0,0.0,0,0.0,0.0,0.0,0,0,0,Not Employed


(7073, 21)

In [90]:
# df_main = dftemp[["age","gender","race","highest_grade","province_current", "tertiary_edu","take_home_pay_feb", "take_home_pay_apr","kept_labour","usual_work"]]




df_main = dftemp[["age","gender","race","highest_grade", "tertiary_edu","province_current","take_home_pay_feb", "take_home_pay_apr","kept_labour","usual_work"]]


### Drop all nans

In [91]:
display(df_main)
df_main_dropped_nans = df_main.dropna()
display(df_main_dropped_nans)

,age,gender,race,highest_grade,tertiary_edu,province_current,take_home_pay_feb,take_home_pay_apr,kept_labour,usual_work
0,39.0,Man,White,Grade 9,No,Gauteng,180.0,0.0,0,Unknown
1,72.0,Woman,Asian/Indian,Grade 0,NaN,KwaZulu-Natal,0.0,0.0,0,Not Employed
2,30.0,Woman,African/Black,Grade 10,Yes,Gauteng,0.0,0.0,0,Not Employed
3,48.0,Woman,African/Black,Grade 10,No,Gauteng,900.0,0.0,0,Elementary occupations
4,49.0,Woman,African/Black,Grade 9,No,Gauteng,0.0,0.0,0,Not Employed
...,...,...,...,...,...,...,...,...,...,...
7068,22.0,Man,African/Black,Grade 11,No,Northern Cape,0.0,0.0,0,Not Employed
7069,24.0,Woman,Coloured,Grade 10,No,Western Cape,0.0,0.0,0,Not Employed
7070,36.0,Woman,African/Black,NTC 3,No,North West,0.0,0.0,0,Not Employed
7071,20.0,Man,African/Black,Grade 12,No,North West,0.0,0.0,0,Not Employed


,age,gender,race,highest_grade,tertiary_edu,province_current,take_home_pay_feb,take_home_pay_apr,kept_labour,usual_work
0,39.0,Man,White,Grade 9,No,Gauteng,180.0,0.0,0,Unknown
2,30.0,Woman,African/Black,Grade 10,Yes,Gauteng,0.0,0.0,0,Not Employed
3,48.0,Woman,African/Black,Grade 10,No,Gauteng,900.0,0.0,0,Elementary occupations
4,49.0,Woman,African/Black,Grade 9,No,Gauteng,0.0,0.0,0,Not Employed
5,34.0,Man,African/Black,Grade 9,No,Gauteng,0.0,0.0,0,Not Employed
...,...,...,...,...,...,...,...,...,...,...
7068,22.0,Man,African/Black,Grade 11,No,Northern Cape,0.0,0.0,0,Not Employed
7069,24.0,Woman,Coloured,Grade 10,No,Western Cape,0.0,0.0,0,Not Employed
7070,36.0,Woman,African/Black,NTC 3,No,North West,0.0,0.0,0,Not Employed
7071,20.0,Man,African/Black,Grade 12,No,North West,0.0,0.0,0,Not Employed


### Now this is the dataframe of all people who kept their jobs, and have usable data

In [92]:
# df_main_kept_labour = df_main_diff[(df_main_diff['kept_labour']==1) & (df_main_diff['salary_change']!="None")]


df_main_kept_labour = df_main_dropped_nans[(df_main_dropped_nans['kept_labour']==1)]

print(list(df_main_kept_labour))

display(df_main_kept_labour)


['age', 'gender', 'race', 'highest_grade', 'tertiary_edu', 'province_current', 'take_home_pay_feb', 'take_home_pay_apr', 'kept_labour', 'usual_work']


,age,gender,race,highest_grade,tertiary_edu,province_current,take_home_pay_feb,take_home_pay_apr,kept_labour,usual_work
15,52.0,Woman,Coloured,Grade 12,No,Gauteng,25000.0,25000.0,1,Technicians and associate professionals
20,40.0,Woman,African/Black,Grade 11,No,Gauteng,3500.0,3500.0,1,Technicians and associate professionals
41,37.0,Woman,African/Black,Grade 12,Yes,Gauteng,4600.0,4600.0,1,Professionals
44,45.0,Woman,African/Black,Grade 12,Yes,Gauteng,4000.0,4200.0,1,Service and sales workers
45,39.0,Woman,African/Black,Grade 11,Yes,Gauteng,3500.0,7200.0,1,Professionals
...,...,...,...,...,...,...,...,...,...,...
7001,43.0,Woman,African/Black,Grade 9,No,Eastern Cape,3200.0,3200.0,1,Elementary occupations
7016,29.0,Man,African/Black,Grade 10,No,North West,4400.0,4500.0,1,Service and sales workers
7018,30.0,Woman,African/Black,Grade 12,Yes,Free State,24000.0,12000.0,1,Professionals
7021,27.0,Woman,African/Black,Grade 9,No,Limpopo,1500.0,2800.0,1,Unknown


### Calculate and store salary difference

In [93]:

diff = df_main_kept_labour['take_home_pay_apr'] - df_main_kept_labour['take_home_pay_feb']

display(diff)

perc = (diff/df_main_kept_labour['take_home_pay_feb'])* 100

perc = np.nan_to_num(perc)
# display(perc)

df_main_diff = df_main_kept_labour.copy()

df_main_diff['take_home_pay_difference'] = perc

15          0.0
20          0.0
41          0.0
44        200.0
45       3700.0
         ...   
7001        0.0
7016      100.0
7018   -12000.0
7021     1300.0
7038        0.0
Length: 705, dtype: float64

### Removing outliers

In [163]:
print("Min percentage change before: ",min(df_main_diff['take_home_pay_difference']))
print("Max percentage change before: ", max(df_main_diff['take_home_pay_difference']))

# Removing Income outliers
df_no_outliers = df_main_diff[df_main_diff["take_home_pay_difference"]<np.percentile(a = df_main_diff["take_home_pay_difference"], q =98)]


print("\n\n\nMin percentage change after: ",min(df_no_outliers['take_home_pay_difference']))
print("Max percentage change after: ",max(df_no_outliers['take_home_pay_difference']))

Min percentage change before:  -100.0
Max percentage change before:  1.7976931348623157e+308



Min percentage change after:  -100.0
Max percentage change after:  135.29411764705884


In [164]:
df_no_outliers

,age,gender,race,highest_grade,tertiary_edu,province_current,take_home_pay_feb,take_home_pay_apr,kept_labour,usual_work,take_home_pay_difference
15,52.0,Woman,Coloured,Grade 12,No,Gauteng,25000.0,25000.0,1,Technicians and associate professionals,0.000000
20,40.0,Woman,African/Black,Grade 11,No,Gauteng,3500.0,3500.0,1,Technicians and associate professionals,0.000000
41,37.0,Woman,African/Black,Grade 12,Yes,Gauteng,4600.0,4600.0,1,Professionals,0.000000
44,45.0,Woman,African/Black,Grade 12,Yes,Gauteng,4000.0,4200.0,1,Service and sales workers,5.000000
45,39.0,Woman,African/Black,Grade 11,Yes,Gauteng,3500.0,7200.0,1,Professionals,105.714286
...,...,...,...,...,...,...,...,...,...,...,...
7001,43.0,Woman,African/Black,Grade 9,No,Eastern Cape,3200.0,3200.0,1,Elementary occupations,0.000000
7016,29.0,Man,African/Black,Grade 10,No,North West,4400.0,4500.0,1,Service and sales workers,2.272727
7018,30.0,Woman,African/Black,Grade 12,Yes,Free State,24000.0,12000.0,1,Professionals,-50.000000
7021,27.0,Woman,African/Black,Grade 9,No,Limpopo,1500.0,2800.0,1,Unknown,86.666667


### Determine if salary changed and create new variable for this

In [232]:
df_newvars = df_no_outliers.copy()

def salary_change(diff):
    if diff==0:
        return "unchanged"
    else:
        return "changed"
    

def salary_diff(diff):
    if diff==0:
        return "unchanged"
    elif diff>0:
        return "increased"
    elif diff<0:
        return "decreased"
    
    
def salary_decreased(diff):
    if diff<0:
        return "decreased"
    else:
        return "didn't decrease"
    
    
def salary_inc_dec(diff):
    if diff<0:
        return "decreased"
    elif diff>0:
        return "increased"
    else:
        return None
    

def check_youth(age):
    if age<25: return 1
    else: return 0 

def check_male(gender): 
    if gender == "Man": return 1
    else: return 0
    
def check_female(gender): 
    if gender == "Woman": return 1
    else: return 0
    
def check_race(race):
    if race == "African/Black": return "Black"
    elif race == "Asian/Indian": return "AsInd"
    else: return race

def check_matric(highest_grade):
    if highest_grade == "Grade 12": return 1
    else: return 0

def check_tertedu(tertiary_edu):
    if tertiary_edu == "Yes" : return 1 
    else: return 0 
    
def check_work(work):
    new_work = work.replace(" ", "_")
    new_work = new_work.replace("-", "_")
    new_work = new_work.replace(",", "")
    return new_work


XYZ = pd.DataFrame()
XYZ["Age"] = df_newvars["age"]
XYZ["Youth"] = np.vectorize(check_youth)(df_newvars["age"])

XYZ["Race"] = np.vectorize(check_race)(df_newvars["race"])

# XYZ["Race"] = df_newvars["race"]


XYZ["Male"] = np.vectorize(check_male)(df_newvars["gender"])

XYZ["Female"] = np.vectorize(check_female)(df_newvars["gender"])


XYZ["Matric"] = np.vectorize(check_matric)(df_newvars["highest_grade"])
XYZ["Tertiary"] = np.vectorize(check_tertedu)(df_newvars["tertiary_edu"])


XYZ["Province"] = df_newvars["province_current"]


XYZ["Pay_Feb"] = df_newvars["take_home_pay_feb"]
XYZ["Pay_Apr"] = df_newvars["take_home_pay_apr"]

XYZ["Pay_Diff"] = df_newvars['take_home_pay_difference']
XYZ["Salary_Change"] = np.vectorize(salary_diff)(df_newvars['take_home_pay_difference']) 

XYZ["Usual_Work"] = np.vectorize(check_work)(df_newvars['usual_work']) 

#Normalize salaries
XYZ["Pay_Feb"] = XYZ["Pay_Feb"] = (XYZ["Pay_Feb"]-XYZ["Pay_Feb"].min())/ (XYZ["Pay_Feb"].max()-XYZ["Pay_Feb"].min())

XYZ["Pay_Apr"] = XYZ["Pay_Apr"] = (XYZ["Pay_Apr"]-XYZ["Pay_Apr"].min())/ (XYZ["Pay_Apr"].max()-XYZ["Pay_Apr"].min())

# XYZ["Pay_Diff"] = XYZ["Pay_Diff"] = (XYZ["Pay_Diff"]-XYZ["Pay_Diff"].min())/ (XYZ["Pay_Diff"].max()-XYZ["Pay_Diff"].min())

XYZ = XYZ[XYZ["Salary_Change"]=="decreased"]

display(XYZ)

,Age,Youth,Race,Male,Female,Matric,Tertiary,Province,Pay_Feb,Pay_Apr,Pay_Diff,Salary_Change,Usual_Work
69,41.0,0,Black,1,0,1,1,Gauteng,0.090,0.060,-33.333333,decreased,Elementary_occupations
93,32.0,0,Black,1,0,0,0,Gauteng,0.049,0.048,-2.040816,decreased,Service_and_sales_workers
136,42.0,0,Black,0,1,1,1,Gauteng,0.030,0.010,-66.666667,decreased,Elementary_occupations
165,46.0,0,Black,0,1,1,0,Gauteng,0.050,0.004,-92.000000,decreased,Service_and_sales_workers
173,28.0,0,Black,0,1,1,1,Gauteng,0.034,0.000,-100.000000,decreased,Service_and_sales_workers
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6500,38.0,0,Black,1,0,1,0,Mpumalanga,0.054,0.052,-3.703704,decreased,Service_and_sales_workers
6525,32.0,0,Black,0,1,0,0,KwaZulu-Natal,0.012,0.000,-100.000000,decreased,Elementary_occupations
6869,34.0,0,Black,0,1,1,1,Gauteng,0.026,0.024,-7.692308,decreased,Elementary_occupations
6923,38.0,0,Black,0,1,1,1,Gauteng,0.120,0.050,-58.333333,decreased,Professionals


### Encode some variables and get dummies

In [233]:
# df_dummies = pd.get_dummies(df_encoded, columns=["race","highest_grade","province_current", "usual_work"], drop_first=True)

LE = LabelEncoder()

df_dummies = XYZ.copy()


df_dummies = pd.get_dummies(XYZ, columns=["Race","Usual_Work"])

# df_dummies = pd.get_dummies(XYZ, columns=["Race","Province","Usual_Work"], drop_first=True)

# df_dummies["Race"] = LE.fit_transform(df_dummies["Race"])
# df_dummies["Usual_Work"] = LE.fit_transform(df_dummies["Usual_Work"])
# df_dummies["Province"] = LE.fit_transform(df_dummies["Province"])

display(df_dummies)
print(df_dummies.columns)

,Age,Youth,Male,Female,Matric,Tertiary,Province,Pay_Feb,Pay_Apr,Pay_Diff,...,Usual_Work_Managers,Usual_Work_Not_Employed,Usual_Work_Not_applicable,Usual_Work_Plant_and_machine_operators_and_assemblers,Usual_Work_Professionals,Usual_Work_Self_Employed,Usual_Work_Service_and_sales_workers,Usual_Work_Skilled_agricultural_forestry_and_fishery_workers,Usual_Work_Technicians_and_associate_professionals,Usual_Work_Unknown
69,41.0,0,1,0,1,1,Gauteng,0.090,0.060,-33.333333,...,0,0,0,0,0,0,0,0,0,0
93,32.0,0,1,0,0,0,Gauteng,0.049,0.048,-2.040816,...,0,0,0,0,0,0,1,0,0,0
136,42.0,0,0,1,1,1,Gauteng,0.030,0.010,-66.666667,...,0,0,0,0,0,0,0,0,0,0
165,46.0,0,0,1,1,0,Gauteng,0.050,0.004,-92.000000,...,0,0,0,0,0,0,1,0,0,0
173,28.0,0,0,1,1,1,Gauteng,0.034,0.000,-100.000000,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6500,38.0,0,1,0,1,0,Mpumalanga,0.054,0.052,-3.703704,...,0,0,0,0,0,0,1,0,0,0
6525,32.0,0,0,1,0,0,KwaZulu-Natal,0.012,0.000,-100.000000,...,0,0,0,0,0,0,0,0,0,0
6869,34.0,0,0,1,1,1,Gauteng,0.026,0.024,-7.692308,...,0,0,0,0,0,0,0,0,0,0
6923,38.0,0,0,1,1,1,Gauteng,0.120,0.050,-58.333333,...,0,0,0,0,1,0,0,0,0,0


Index(['Age', 'Youth', 'Male', 'Female', 'Matric', 'Tertiary', 'Province',
       'Pay_Feb', 'Pay_Apr', 'Pay_Diff', 'Salary_Change', 'Race_AsInd',
       'Race_Black', 'Race_Coloured', 'Race_White',
       'Usual_Work_Armed_forces_occupations',
       'Usual_Work_Clerical_support_workers',
       'Usual_Work_Craft_and_related_trades_workers',
       'Usual_Work_Elementary_occupations', 'Usual_Work_Managers',
       'Usual_Work_Not_Employed', 'Usual_Work_Not_applicable',
       'Usual_Work_Plant_and_machine_operators_and_assemblers',
       'Usual_Work_Professionals', 'Usual_Work_Self_Employed',
       'Usual_Work_Service_and_sales_workers',
       'Usual_Work_Skilled_agricultural_forestry_and_fishery_workers',
       'Usual_Work_Technicians_and_associate_professionals',
       'Usual_Work_Unknown'],
      dtype='object')


### Store everything done in a new dataframe and drop columns that we don't want in model

In [234]:
firstFucker = smf.ols(formula = "Pay_Diff ~ Usual_Work_Armed_forces_occupations +  Usual_Work_Craft_and_related_trades_workers + Usual_Work_Clerical_support_workers + Usual_Work_Plant_and_machine_operators_and_assemblers + Usual_Work_Professionals + Usual_Work_Service_and_sales_workers + Usual_Work_Self_Employed + Usual_Work_Technicians_and_associate_professionals", data=df_dummies).fit()
firstFucker.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     1.577
Date:                Sat, 19 Jun 2021   Prob (F-statistic):              0.136
Time:                        00:20:34   Log-Likelihood:                -769.49
No. Observations:                 158   AIC:                             1557.
Df Residuals:                     149   BIC:                             1585.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================
                                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
Intercept                                               -48.9375      5.135     -9.530      0.000     -59.085     -38.790
Usual_Work_Armed_forces_occupations                      30.1875     32.882      0.918      0.360     -34.787      95.162
Usual_Work_Craft_and_related_trades_workers               3.3547     12.579      0.267      0.790     -21.501      28.210
Usual_Work_Clerical_support_workers                       1.9417      9.049      0.215      0.830     -15.940      19.823
Usual_Work_Plant_and_machine_operators_and_assemblers     6.5553     11.483      0.571      0.569     -16.135      29.245
Usual_Work_Professionals                                  5.4832      8.386      0.654      0.514     -11.087      22.054
Usual_Work_Service_and_sales_workers                     18.0883      7.262      2.491      0.014       3.738      32.439
Usual_Work_Self_Employed                                 38.9375     32.882      1.184      0.238     -26.037     103.912
Usual_Work_Technicians_and_associate_professionals      -10.3019      9.833     -1.048      0.296     -29.733       9.129
==============================================================================
Omnibus:                       18.109   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.426
Skew:                          -0.560   Prob(JB):                      0.00200
Kurtosis:                       2.205   Cond. No.                         13.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [249]:
second = smf.ols(formula = "Pay_Diff ~ Male", data=df_dummies).fit()
second.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     4.425
Date:                Sat, 19 Jun 2021   Prob (F-statistic):             0.0370
Time:                        00:24:04   Log-Likelihood:                -773.70
No. Observations:                 158   AIC:                             1551.
Df Residuals:                     156   BIC:                             1558.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -48.1131      3.475    -13.846      0.000     -54.977     -41.249
Male          10.9822      5.221      2.104      0.037       0.670      21.295
==============================================================================
Omnibus:                       23.682   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.564
Skew:                          -0.521   Prob(JB):                      0.00187
Kurtosis:                       2.094   Cond. No.                         2.51
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [254]:
poes = smf.ols(formula = "Pay_Diff ~ Male + Race_Black", data=df_dummies).fit()
poes.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     2.343
Date:                Sat, 19 Jun 2021   Prob (F-statistic):             0.0994
Time:                        00:24:55   Log-Likelihood:                -773.55
No. Observations:                 158   AIC:                             1553.
Df Residuals:                     155   BIC:                             1562.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -45.6935      5.740     -7.960      0.000     -57.033     -34.354
Male          11.0514      5.234      2.111      0.036       0.711      21.391
Race_Black    -3.2261      6.084     -0.530      0.597     -15.244       8.792
==============================================================================
Omnibus:                       24.156   Durbin-Watson:                   2.017
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.320
Skew:                          -0.508   Prob(JB):                      0.00211
Kurtosis:                       2.083   Cond. No.                         4.22
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [243]:
third = smf.ols(formula = "Pay_Diff ~ Usual_Work_Service_and_sales_workers", data=df_dummies).fit()
third.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     7.916
Date:                Sat, 19 Jun 2021   Prob (F-statistic):            0.00553
Time:                        00:22:01   Log-Likelihood:                -772.00
No. Observations:                 158   AIC:                             1548.
Df Residuals:                     156   BIC:                             1554.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                              -47.4504      2.969    -15.983      0.000     -53.315     -41.586
Usual_Work_Service_and_sales_workers    16.6012      5.900      2.814      0.006       4.946      28.256
==============================================================================
Omnibus:                       23.494   Durbin-Watson:                   1.942
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               14.001
Skew:                          -0.581   Prob(JB):                     0.000911
Kurtosis:                       2.118   Cond. No.                         2.48
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [244]:
fourth = smf.ols(formula = "Pay_Diff ~ Youth", data=df_dummies).fit()
fourth.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                   0.07057
Date:                Sat, 19 Jun 2021   Prob (F-statistic):              0.791
Time:                        00:22:40   Log-Likelihood:                -775.87
No. Observations:                 158   AIC:                             1556.
Df Residuals:                     156   BIC:                             1562.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -43.4089      2.698    -16.086      0.000     -48.739     -38.079
Youth          3.1858     11.992      0.266      0.791     -20.503      26.874
==============================================================================
Omnibus:                       32.400   Durbin-Watson:                   1.977
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               14.360
Skew:                          -0.546   Prob(JB):                     0.000762
Kurtosis:                       2.005   Cond. No.                         4.57
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [247]:
best = smf.ols(formula = "Pay_Diff ~ Male + Race_Black + Usual_Work_Service_and_sales_workers", data=df_dummies).fit()
best.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Pay_Diff   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     4.207
Date:                Sat, 19 Jun 2021   Prob (F-statistic):            0.00682
Time:                        00:23:41   Log-Likelihood:                -769.68
No. Observations:                 158   AIC:                             1547.
Df Residuals:                     154   BIC:                             1560.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                              -46.2789      5.624     -8.230      0.000     -57.388     -35.170
Male                                     9.2860      5.164      1.798      0.074      -0.914      19.486
Race_Black                              -7.0357      6.111     -1.151      0.251     -19.109       5.037
Usual_Work_Service_and_sales_workers    16.8304      6.053      2.781      0.006       4.873      28.788
==============================================================================
Omnibus:                       21.244   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.334
Skew:                          -0.529   Prob(JB):                      0.00210
Kurtosis:                       2.133   Cond. No.                         4.38
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""